<a href="https://colab.research.google.com/github/Benjamin-morel/TensorFlow/blob/main/01_classification_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


# Machine Learning Model - MNIST dataset - basic image classification

| | |
|------|------|
| Filename | 01_classification_image.ipynb |
| Author(s) | Benjamin Morel (benjaminmorel27@gmail.com) |
| Date | September 2, 2024 |
| Aim(s) | Build, train and evaluate a neural network machine learning model that classifies images. |
| Version | Python 3.12 |


<br> **!!Read before running!!** <br>
1. Frames have been intentionally hidden. Please reopen them to see the code.
2. Run all and read comments.

---



This Python script builds a **neural network** (NN) and trains it to classify images. The objective is to classify a relatively significant number - 10 - of different images. To achieve this, the neural network defines its own internal parameters during the **training phase** to correctly classify the images according to the **label** provided for each input. Finally, the neural network undergoes a test and **evaluation phase** in which it has to classify unknow images similar to the one it has learned during the training phase, but without knowing the label. The prediction made by the neural network is then compared with the label provided. These steps are shown in the code below.

## 1. Import libraries & prebuilt dataset

**Comments**
<br> First, data correctly labeled is needed to train and test the network. For this purpose, the large **MNIST database** is used [[1]](https://yann.lecun.com/exdb/mnist/). This image database is available in TensorFlow library. Images are 28 pixels by 28 pixels and represent **handwritten digits**. Each image is assigned a value corresponding to a **gray level*** on a gray scale from 0 to 255 (RGB code, where the three primary colors are equal). The use of *Plotly* rather than *Matplotlib* as the library for graph plotting is a preference of the code's author.

In [1]:
pip install -q -U keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf  # machine learning models
import numpy as np # scientific computing
import plotly.express as px # graphing packages
from plotly.subplots import make_subplots # make subplot graphs in plotly
import keras_tuner as kt

print("TensorFlow version:", tf.__version__)

mnist = tf.keras.datasets.mnist # import MNIST dataset (70,000 handwritten digit images of 28x28 pixels)

TensorFlow version: 2.17.0


**Comments**
<br> From the MNIST database, training and test sets are declared as **tensors** (easier handling and the ability to exploit all TensorFlow's functionality). The **training tensor** `x_train` contains pixel values of images. This tensor is composed of 60,000 "sub-tensors" of 28x28 elements. For each 28x28 sub-tensor, the name of the label is given by the `y_train` tensor (integers ranging from 0 to 9). For the **test data**, 10,000 images are used, following the same logic as for the training set. Data are pre-processed by **normalizing** it. The pixel values are now ranged between 0 and 1. <br>

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() # training tensor of 60,000 samples + test tensor of 10,000 samples
x_train, x_test = x_train / 255.0, x_test / 255.0 # standardization, pixel value is now bewteen 0 and 1

fig = px.imshow(x_train[:10, :, :], color_continuous_scale='gray_r', facet_col=0, binary_string=False) # plot an example of the first 10 training data
for i, label in enumerate(y_train[:10]): # label of the training data
    fig.layout.annotations[i]['text'] = 'label: %d' % label
fig.update_layout(margin=dict(l=10, r=10, t=100, b=100), width=1000, height=300)
fig.update_yaxes(visible=False, showticklabels=False), fig.update_xaxes(visible=False, showticklabels=False), fig.update(layout_coloraxis_showscale=False)
fig.show()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## 2. Build the neural network machine learning model

**Comments**
<br> To build the neural network, the powerful deep learning API *KERAS* is used in *TensorFlow*. Layers are created and added to the *TensorFlow* model. The first layer `tf.keras.layers.Flatten` converts a two-dimensional array (28x28 pixels) to a one-dimensional array (784 pixels). No parameters are learnt in this layer, only a **reformatting of data** is done. Then a **densily-connected layer** with 128 neurons is created with `tf.keras.layers.Dense`. The ReLu function is used as an **activation function**. A dropout layer in *TensorFlow* is used to prevent **overfitting**. The user defined a dropout rate `DR` which determines the probability of any given neuron being excluded temporaly from the neural network [[2]](https://www.scaler.com/topics/dropout-tensorflow/). At each training batch iteration, random neurons are desactivated according to the dropout rate. Therefore, the model must learn redundant representations and rely on something other than specific neurons for accurate predictions. Finally a layer of 10 neurons is defined as **output layer** in order to return a probability for each digits (0 to 9).



In [4]:
DR = 0.2

model = tf.keras.models.Sequential() # stack layers - each layer has 1 input tensor and 1 output tensor
model.add(tf.keras.layers.Flatten(input_shape=(28, 28))) # input layer
model.add(tf.keras.layers.Dense(128, activation='relu')) # hidden layer
model.add(tf.keras.layers.Dropout(DR))
model.add(tf.keras.layers.Dense(10)) # classification layer

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



## 3. Train the model

**Comments**
<br> During this step, the neural network **ajusts** and **optimizes** the thousands of parameters it has for each neuron input. To do this, training data, images and label associated for each, is used. **Parameter optimization** is based on minimization of the loss function `SparseCategoricalCrossentropy` which computes the cross-entropy loss between true labels and predicted labels. Optimization is performed using the **gradient descent algorithm** *ADAM*. The model has 5 iterations for the learning phase.

In [5]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy']) # use a stochastic gradient descent to minimize the loss function
model.fit(x_train, y_train, epochs=5)
model.summary()

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.8594 - loss: 0.4853
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.9557 - loss: 0.1511
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.9671 - loss: 0.1111
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9726 - loss: 0.0892
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9773 - loss: 0.0721


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 305,312 (1.16 MB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 203,542 (795.09 KB)

## 4. Evaluate the model



**Comments**
<br> Once the model has been trained, it can be tested and **evaluated** by comparing the predictions it makes from the test set with its true values. It is also possible to study the probabilities it associates with each digit for a particular image. To do this, the values calculated at the output of the hidden layer are converted into probability using the ***softmax* function**. For each digit, a probability is associated and the model concludes by considering the **highest probability**.

In [6]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=1)
print(" --------------------------------------------- \n", 100*round(test_acc, 3) , "% of the test set is correctly predicted \n", "---------------------------------------------\n")
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9738 - loss: 0.0852
 --------------------------------------------- 
 97.8 % of the test set is correctly predicted 
 ---------------------------------------------



**Comments**
<br> It is possible to explore the different inputs of the test set and check what the model has predicted. To do this, the user enters the index of the image `i`for which the user wishes to know the prediction. The probabilities calculated by the model are displayed in a bar chart. The model concludes on the outfit with the highest probability. <br>

In [7]:
i = 1

In [8]:
def plot_value_array(i, predictions_array, true_label): # plot a bar chart with the probability value computed according to the label
  fig = px.bar(100*predictions_array, width=800, height=400, text_auto='.2f')
  fig.update_xaxes(title="Digit", tickvals=np.arange(10), ticktext=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]), fig.update_layout(showlegend=False), fig.update_yaxes(title="Probability (%)")
  fig.show()

predictions = probability_model.predict(x_test) # predict the label of each test sample according to the probability computed with softmax function
predictions[i] # first prediction
prediction_label = np.argmax(predictions[i]) # get the label of the max probability
true_label = y_test[i]
print("Prediction: ", prediction_label)
print("Label: ", true_label)

if prediction_label == true_label:
  print("\nGREAT!!")
else:
  print("\nBad prediction!")

plot_value_array(i, predictions[i],  true_label)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Prediction:  2
Label:  2

GREAT!!


## 5. Go further with the Fashion MNIST dataset

**Comments**
<br> Now, the neural network has to classify among 70,000 28x28 labeled **clothe images** from the Fashion MNIST database. There are 10 different items:
*   T-shirt (label value = 0)
*   Trouser (=1)
*   Pullover (=2)
*   Dress (=3)
*   Coat (=4)
*   Sandal (=5)
*   Shirt (=6)
*   Sneaker (=7)
*   Bag (=8)
*   Ankle boot (=9)
<br>

In [9]:
fashion_mnist = tf.keras.datasets.fashion_mnist # import fashion MNIST dataset

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data() # create training and test tensors
train_images, test_images = train_images / 255.0, test_images / 255.0 # pre-processing training and test data

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] # clothe labels present in the dataset

fig = px.imshow(train_images[:12, :, :], color_continuous_scale='gray_r', facet_col=0, binary_string=False) # plot an example of the first 12 training data
for i, label in enumerate(train_labels[:12]): # label of the training data
    fig.layout.annotations[i]['text'] = '%s' % class_names[label]
fig.update_layout(margin=dict(l=10, r=10, t=100, b=100), width=1000, height=300)
fig.update_yaxes(visible=False, showticklabels=False), fig.update_xaxes(visible=False, showticklabels=False), fig.update(layout_coloraxis_showscale=False)
fig.show()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


**Comments**
<br> Same model and loss function minimization as the previous example. The training phase is longer than the previous one. The test phase computes the accuracy of the neural network.

In [10]:
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(tf.keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [11]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [12]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [13]:
tuner.search(train_images, train_labels, epochs=20, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 14s]
val_accuracy: 0.8600833415985107

Best val_accuracy So Far: 0.8914999961853027
Total elapsed time: 00h 19m 48s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



In [14]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, epochs=30, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.7834 - loss: 0.6133 - val_accuracy: 0.8542 - val_loss: 0.3958
Epoch 2/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.8604 - loss: 0.3789 - val_accuracy: 0.8740 - val_loss: 0.3544
Epoch 3/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.8786 - loss: 0.3269 - val_accuracy: 0.8674 - val_loss: 0.3545
Epoch 4/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.8852 - loss: 0.3086 - val_accuracy: 0.8783 - val_loss: 0.3327
Epoch 5/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.8926 - loss: 0.2865 - val_accuracy: 0.8844 - val_loss: 0.3173
Epoch 6/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.9024 - loss: 0.2644 - val_accuracy: 0.8835 - val_loss: 0.3273
Epoch 7/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9053 - loss: 0.2549 - val_accuracy: 0.8797 - val_loss: 0.3386
Epoch 8/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.9126 - loss: 0

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

In [16]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)
print(" --------------------------------------------- \n", 100*round(test_acc, 3) , "% of the test set is correctly predicted \n", "---------------------------------------------\n")
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
model.summary()

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8814 - loss: 0.4562
 --------------------------------------------- 
 88.3 % of the test set is correctly predicted 
 ---------------------------------------------



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 480)                 │         376,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           4,810 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,144,832 (4.37 MB)

 Trainable params: 381,610 (1.46 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 763,222 (2.91 MB)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)
print(" --------------------------------------------- \n", 100*round(test_acc, 3) , "% of the test set is correctly predicted \n", "---------------------------------------------\n")
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
model.summary()

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9405 - loss: 0.1811
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.9449 - loss: 0.1560
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.9468 - loss: 0.1479
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9485 - loss: 0.1402
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9527 - loss: 0.1267
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9519 - loss: 0.1261
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9524 - loss: 0.1280
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9566 - loss: 0.1192
Epoch 9/10
  44/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9652 - loss: 0.0848

**Comments**
<br> It is possible to explore the different inputs of the test set and check what the model has predicted. To do this, the user enters the index of the image `i`for which the user wishes to know the prediction. The probabilities calculated by the model are displayed in a bar chart. The model concludes on the outfit with the highest probability. Try `i=12`to study a bad prediction case. <br>


In [ ]:
i = 12

In [ ]:
def plot_value_array(i, predictions_array, true_label): # plot a bar chart with the probability value computed according to the label
  true_label = true_label[i]
  fig = px.bar(100*predictions_array, width=800, height=400)
  fig.update_xaxes(title="Outfit", tickvals=np.arange(10), ticktext=class_names), fig.update_layout(showlegend=False), fig.update_yaxes(title="Probability (%)")
  fig.show()

predictions = probability_model.predict(test_images) # predict the label of each test sample according to the probability computed with softmax function
predictions[i] # first prediction
prediction_label = np.argmax(predictions[i]) # get the label of the max probability
true_label = test_labels[i]
print("Prediction: ", class_names[prediction_label] , "(",round(100*np.max(predictions[i]),1), "%)")
print("Label: ", class_names[true_label])

if prediction_label == true_label:
  print("\nGREAT!!")
else:
  print("\nBad prediction!")

plot_value_array(i, predictions[i],  test_labels)

**Comments**
<br> Finally, in order to done a more critical analysis of the neural network, the confusion matrix is calculated to identify confusions requiring further training. Here, T-shirt, shirt and sweater are 3 clothes that are sometimes confused. The shapes of these 3 clothes are very similar. It would be a good idea to perform a second training phase for these clothes by adding additional features to these 3 clothes (sleeve size, shirt buttons, texture, etc.).

In [ ]:
confusion_mat = tf.math.confusion_matrix(test_labels,np.argmax(predictions, axis=1))
fig = px.imshow(confusion_mat, x = class_names, y = class_names, text_auto=True)
fig.update(layout_coloraxis_showscale=False)
fig.show()

## 6. Go further again...

**Comments**
<br> A final question that is legitimate to ask is: “Would doubling the number of hidden layers increase the accuracy of the model for the same learning and training set?” For this, the same Fashion MNIST dataset is taken and the neural network is modified to add a 2nd hidden layer of 128 neurons. Computation time increases by 9% compared to the single hidden layer model, and accuracy by only 0.35%. Adding more layers of neurons is not a serious solution in this case.

In [ ]:
DR = 0.2
model = tf.keras.models.Sequential() # stack layers - each layer has 1 input tensor and 1 output tensor
model.add(tf.keras.layers.Flatten(input_shape=(28, 28))) # input layer
model.add(tf.keras.layers.Dense(128, activation='relu')) # hidden layer
model.add(tf.keras.layers.Dense(128, activation='relu')) # hidden layer
model.add(tf.keras.layers.Dropout(DR))
model.add(tf.keras.layers.Dense(10)) # classification layer

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)
print(" --------------------------------------------- \n", 100*round(test_acc, 3) , "% of the test set is correctly predicted \n", "---------------------------------------------\n")